In [1]:
import pandas as pd
import numpy as np

# creating the data set

In [2]:
d = {}
d['Participant'] = ['Alice','Bob','Cary','Doug','Edna']
d['Tax Rate'] = [3,4,3,2,1]
d['Fee'] = [4,3,5,1,1]
d['Interest Rate'] = [3,5,3,3,3]
d['Quantity Limit'] = [2,1,3,3,2]
d['Price Limit'] = [1,1,3,2,3]
df = pd.DataFrame(d)

# transforming the dataset to a matrix

In [3]:
df_mtx=df.as_matrix(columns=df.columns[1:])
df_mtx

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[3, 4, 3, 2, 1],
       [4, 3, 5, 1, 1],
       [3, 5, 3, 3, 3],
       [2, 1, 3, 3, 2],
       [1, 1, 3, 2, 3]], dtype=int64)

# similarity matrix

In [4]:
sim_mtx=np.zeros(df_mtx.shape)
def similarity_matrix():
    for i in range(len(df_mtx)):
        for j in range(len(df_mtx)):
            sim_mtx[i][j]=-sum((df_mtx[i]-df_mtx[j])**2)
    for k in range(len(df_mtx)):
        sim_mtx[k][k]=sim_mtx.min()
    return sim_mtx  
similarity_matrix()


array([[-22.,  -7.,  -6., -12., -17.],
       [ -7., -22., -17., -17., -22.],
       [ -6., -17., -22., -18., -21.],
       [-12., -17., -18., -22.,  -3.],
       [-17., -22., -21.,  -3., -22.]])

# Responsibilty matrix

In [5]:
res_mtx=np.zeros(df_mtx.shape)
def responsibilty_matrix():
    for i in range(len(df_mtx)):
        for j in range(len(df_mtx)):
            res_mtx[i][j]=similarity_matrix()[i][j]-np.delete(similarity_matrix()[i],j).max()
    return(res_mtx)
responsibilty_matrix()

array([[-16.,  -1.,   1.,  -6., -11.],
       [ 10., -15., -10., -10., -15.],
       [ 11., -11., -16., -12., -15.],
       [ -9., -14., -15., -19.,   9.],
       [-14., -19., -18.,  14., -19.]])

# Availibility matrix

In [6]:
avail_mtx=np.zeros(df_mtx.shape)
def availibility_matrix():
    for i in range(len(df_mtx)):
        for j in range(len(df_mtx)):
            if i==j:
                avail_mtx[i][j]=sum([k[j] for k in responsibilty_matrix() if k[j]>0])
            else:
                avail_mtx[i][j]=min(0,sum(list(set([k[j] for k in responsibilty_matrix() if k[j]>0]+[responsibilty_matrix()[j][j],])-set([responsibilty_matrix()[i][j],]))))
            
    return(avail_mtx)
availibility_matrix()

array([[ 21., -15., -16.,  -5., -10.],
       [ -5.,   0., -15.,  -5., -10.],
       [ -6., -15.,   1.,  -5., -10.],
       [  0., -15., -15.,  14., -19.],
       [  0., -15., -15., -19.,   9.]])

#  mentioning the parameters and calculating Criterion matrix

In [24]:
damping=0.8 #noise for converge
n=100 #no. of iterations
def criterion_matrix():
    avail_mtx=np.zeros(df_mtx.shape)
    res_mtx=np.zeros(df_mtx.shape)
    for i in range(n):
        similarity_matrix()
        if i==0:
            responsibilty_matrix()
            availibility_matrix()
        else:
            res_mtx=damping*res_mtx +(1-damping)*responsibilty_matrix()
            avail_mtx=damping*avail_mtx+(1-damping)*availibility_matrix()
    c_mtx=res_mtx+avail_mtx
    return(c_mtx)

# forming clusters 

In [26]:
clusters=[np.argmax(i) for i in criterion_matrix()]
clusters

[0, 0, 0, 3, 3]